# Run a test calculation with `a-bam-p`/`acd-1-p` to find gaps in our workflow...

In [4]:
%load_ext autoreload
%autoreload 2

import os as os
import glob as glob
import numpy as np

import parmed as pmd
from paprika import align
from paprika import dummy
from paprika import build

import paprika
print(paprika.__version__)

import smirnovert
print(smirnovert.__version__)

from smirnovert.convert import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
a8e4be9
a8e4be9


In [5]:
prefix = './benchmarksets/input_files/'
suffix = '/prmtop-rst7/'
benchmarksets = [
    'cd-set1'
    ]

systems = []

for system in benchmarksets:
    for index, file in enumerate(sorted(glob.glob(prefix + system + suffix + 'acd-1-p.prmtop'))):
        systems.append(file)

## Load in the reference coordinates and SMIRNOFF99Frosst parameters

In [6]:
destination = './benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/'
name = 'smirnoff'
reference_prmtop = 'acd-1-p.prmtop'
reference_inpcrd = 'acd-1-p.rst7'
host_resname = 'MGO'
guest_resname = 'MOL'

nonunique = convert(destination, name, 
    reference_prmtop, reference_inpcrd, 
    host_resname, guest_resname,
               debug=True)

INFO:root:Creating ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/smirnoff.pdb with CONECT records...
DEBUG:root:PDB file written by cpptraj.
INFO:root:Pruning water-water CONECT records...
DEBUG:root:First water residue = 151
DEBUG:root:Found first water CONECT entry at line = 6304
INFO:root:Splitting topology into components...
INFO:root:Creating a combined topology for the host and guest molecules...
INFO:root:Writing a `mol2` for the host molecule...
DEBUG:root:MOL2 file written by cpptraj.
INFO:root:Writing a `mol2` for the host molecule...
DEBUG:root:MOL2 file written by cpptraj.
INFO:root:Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/MGO.mol2 to SYBYL atom types via Antechamber...
DEBUG:root:MOL2 file written by antechamber.
INFO:root:Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/MOL.mol2 to SYBYL atom types via Antechamber...
DEBUG:root:MOL2 file written by antechamber.
INFO:root:Extracting water and 

In [7]:
from smirnovert.amber import create_mapping

In [8]:
host_mapping, guest_mapping = create_mapping(nonunique, host_resname, guest_resname)

In [9]:
from smirnovert.amber import remap_atom_types

In [10]:
parm = pmd.amber.LoadParm(destination + 'hg.prmtop')

In [11]:
remap_atom_types(parm, host_resname, host_mapping, guest_resname, guest_mapping, destination)

Assigning 0 C1 1 → C0
Assigning 0 H1 2 → H0
Assigning 0 O1 3 → O0
Assigning 0 C2 4 → C1
Assigning 0 H2 5 → H1
Assigning 0 O2 6 → O1
Assigning 0 HO2 7 → H2
Assigning 0 C3 8 → C2
Assigning 0 H3 9 → H3
Assigning 0 O3 10 → O2
Assigning 0 HO3 11 → H4
Assigning 0 C4 12 → C3
Assigning 0 H4 13 → H5
Assigning 0 C5 14 → C4
Assigning 0 H5 15 → H6
Assigning 0 O5 16 → O3
Assigning 0 C6 17 → C5
Assigning 0 H61 18 → H7
Assigning 0 H62 19 → H8
Assigning 0 O6 20 → O4
Assigning 0 HO6 21 → H9
Assigning 1 C1 22 → C0
Assigning 1 H1 23 → H0
Assigning 1 O1 24 → O0
Assigning 1 C2 25 → C1
Assigning 1 H2 26 → H1
Assigning 1 O2 27 → O1
Assigning 1 HO2 28 → H2
Assigning 1 C3 29 → C2
Assigning 1 H3 30 → H3
Assigning 1 O3 31 → O2
Assigning 1 HO3 32 → H4
Assigning 1 C4 33 → C3
Assigning 1 H4 34 → H5
Assigning 1 C5 35 → C4
Assigning 1 H5 36 → H6
Assigning 1 O5 37 → O3
Assigning 1 C6 38 → C5
Assigning 1 H61 39 → H7
Assigning 1 H62 40 → H8
Assigning 1 O6 41 → O4
Assigning 1 HO6 42 → H9
Assigning 2 C1 43 → C0
Assigning 

## Now re-load in the new structure

In [12]:
hg = pmd.load_file(destination + 'smirnoff-unique.prmtop', destination + 'hg.inpcrd', structure=True)

## Align

In [13]:
# Find the guest atoms to align by reading first line in /data/nhenriksen/projects/cds/wat6/bgbg-tip3p/pdbs
hg = align.zalign(hg, ':MOL@C4', ':MOL@N1', save=True, filename=destination + 'hg-aligned.pdb')

INFO:root:Moving :MOL@C4 (1 atoms) to the origin...
INFO:root:Aligning :MOL@N1 (1 atoms) with the z axis...
INFO:root:Saved aligned coordinates to ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/hg-aligned.pdb


## Add dummy atoms

In [14]:
hg = pmd.load_file(destination + 'hg-aligned.pdb', structure=True)
# print(hg.atoms[-1].number)

hg = dummy.add_dummy(hg, residue_name='DM1', z=-6.000)
hg = dummy.add_dummy(hg, residue_name='DM2', z=-9.000)
hg = dummy.add_dummy(hg, residue_name='DM3', z=-11.200, y=2.200)

dummy.write_dummy_frcmod(path = destination)
dummy.write_dummy_mol2(path = destination, residue_name='DM1', filename='DM1.mol2')
dummy.write_dummy_mol2(path = destination, residue_name='DM2', filename='DM2.mol2')
dummy.write_dummy_mol2(path = destination, residue_name='DM3', filename='DM3.mol2')

hg.write_pdb(destination + 'hg-aligned-dum.pdb', renumber=False)

## Solvate

In [17]:
base_name = 'smirnoff'
with open(destination + 'tleap.in', 'w') as file:
    tleap_header = f'''\n
source leaprc.water.tip3p
loadamberparams {base_name}-unique.frcmod
{host_resname} = loadmol2 {base_name}-{host_resname}-unique.mol2
{guest_resname} = loadmol2 {base_name}-{guest_resname}-unique.mol2

DM1 = loadmol2 DM1.mol2
DM2 = loadmol2 DM2.mol2
DM3 = loadmol2 DM3.mol2
loadamberparams dummy.frcmod

model = loadpdb {str('hg-aligned-dum.pdb')}
bond model.1.O1 model.2.C4
bond model.2.O1 model.3.C4
bond model.3.O1 model.4.C4
bond model.4.O1 model.5.C4
bond model.5.O1 model.6.C4
bond model.6.O1 model.1.C4


'''
    file.write(tleap_header)
#     for line in bonds_between_monomers:
#         file.write(line + '\n')

In [23]:
destination = './benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/'

build.solvate(tleap_file='tleap.in', pdb_file='hg-aligned-dum.pdb', pbc_type='rectangular', 
              buffer_target=2000,
              output_prefix='hg-solvated', path=destination)

DEBUG:root:Cycle	Buffer Waters Target
DEBUG:root:00	 1 748 (2000)
DEBUG:root:01	 11 1534 (2000)
DEBUG:root:02	 21 2329 (2000)
DEBUG:root:03	 16 1936 (2000)
DEBUG:root:04	 16 1988 (2000)
DEBUG:root:05	 16 1988 (2000)
DEBUG:root:06	 16 2000 (2000)
